<img src="animated.gif" width=500>

# How to make an animated GIF using a canvas

This notebook demonstrates how to capture image arrays from a jp_doodle canvas
and save the collected images as an animated GIF.

For the illustration we develop an animation which projects a hypercube into three
dimensions using a projectiom matrix.

In [1]:
import numpy as np

In [2]:
def projection_array(t, delta=0.01, gamma=0.0111):
    "projection matrix for a time point."
    alpha = delta * t
    beta = 1.0 + gamma * t
    delta = 2.0 + (delta + gamma) * t
    ca, sa = np.sin(alpha), np.cos(alpha)
    cb, sb = np.sin(beta), np.cos(beta)
    cd, sd= np.sin(delta), np.cos(delta)
    return np.array([
        [sa, -ca, 0, sd],
        [ca,  sa, sd, 0],
        [cd,   0, sb, cb],
    ]).transpose()

#projection_array(100)

In [3]:
# hypercube vertices
vertices4 = np.array([
    [i, j, k, l]
    for i in [-1,1]
    for j in [-1,1]
    for k in [-1,1]
    for l in [-1,1]
])
#vertices4

In [4]:
# hypercube edges
from numpy.linalg import norm
edges = []
for (i, v) in enumerate(vertices4):
    for (j, w) in enumerate(vertices4):
        if i > j and norm(v - w) == 2:
            edges.append((i,j))
#edges

In [5]:
def project_vertices(t):
    "project 4d vertices into 3d at a time point."
    P = projection_array(t)
    return vertices4.dot(P)

#project_vertices(555)

# Create a canvas for the animation

In [6]:
from jp_doodle.nd_frame import swatch3d
s = swatch3d(pixels=700, model_height=6)

def draw_hcube(t=220):
    s.reset()
    s.frame_rect((-7, -7, -7), 10, 10, color="yellow")
    vertices = project_vertices(t)
    for (i, j) in edges:
        s.line(vertices[i], vertices[j], color="#e94", lineWidth=5)
    s.orbit_all(3)
        
draw_hcube()

DualCanvasWidget(status='deferring flush until render')

# Draw an animation on the canvas and capture images for each frame

In [7]:
image_arrays = []

def array_callback(array):
    
    image_arrays.append(array)

import time
for t in range(200):
    with s.in_canvas.delay_redraw():
        draw_hcube(t)
    #array = s.in_canvas.pixels_array()
    #image_arrays.append(array)
    s.in_canvas.pixels_array_async(array_callback)
    time.sleep(0.1)

In [8]:
# View one of the saved images
from jp_doodle.array_image import show_array
show_array(image_arrays[5])

DualCanvasWidget(status='deferring flush until render')

# Save the image sequence as an animated GIF using `imageio`

In [9]:
import imageio

In [10]:
exportname = "animated.gif"
imageio.mimsave(exportname, image_arrays, format='GIF', duration=0.1)

<img src="animated.gif" width=500>